In [2]:
import re
import os
import csv

In [3]:
#dir_path = os.path.dirname(os.path.realpath(__file__))
dir_path = os.path.abspath('.')
print(dir_path)
debug = True

/home/yihan/Framework/data


In [4]:
#os.listdir("cmu-wafer/normal")

In [5]:
def parse_filename(filename: str):
    return tuple(re.split("_|\.", filename))

# example:
if debug:
    print(parse_filename('1551_05.12'))

('1551', '05', '12')


In [6]:
# How to use
#list(map(parse_filename, _2))

In [7]:
# 반복문 사용법

for filename in os.listdir("cmu-wafer/normal"):
    with open(os.path.join("cmu-wafer/normal", filename)) as f:
        print(f.readlines())
    break

['    1\t2\t\n', '    2\t2\t\n', '    3\t2\t\n', '    4\t2\t\n', '    5\t2\t\n', '    6\t2\t\n', '    7\t2\t\n', '    8\t2\t\n', '    9\t2\t\n', '   10\t2\t\n', '   11\t2\t\n', '   12\t2\t\n', '   13\t2\t\n', '   14\t2\t\n', '   15\t2\t\n', '   16\t2\t\n', '   17\t2\t\n', '   18\t2\t\n', '   19\t2\t\n', '   20\t2047\t\n', '   21\t2047\t\n', '   22\t2047\t\n', '   23\t2047\t\n', '   24\t2047\t\n', '   25\t2047\t\n', '   26\t2047\t\n', '   27\t2047\t\n', '   28\t2047\t\n', '   29\t2047\t\n', '   30\t2047\t\n', '   31\t2047\t\n', '   32\t2047\t\n', '   33\t2047\t\n', '   34\t3\t\n', '   35\t2\t\n', '   36\t2\t\n', '   37\t2\t\n', '   38\t2\t\n', '   39\t2\t\n', '   40\t2\t\n', '   41\t2\t\n', '   42\t2\t\n', '   43\t2\t\n', '   44\t2\t\n', '   45\t2\t\n', '   46\t2\t\n', '   47\t2\t\n', '   48\t2\t\n', '   49\t2\t\n', '   50\t1026\t\n', '   51\t1181\t\n', '   52\t1188\t\n', '   53\t1191\t\n', '   54\t1196\t\n', '   55\t1201\t\n', '   56\t1205\t\n', '   57\t1210\t\n', '   58\t1219\t\n', ' 

In [9]:
# 단일 파일 읽기 함수 - normal
def read_data(datatype: str, filename: str):
    
    assert datatype == "normal" or datatype == "abnormal"
    label = True if datatype == "normal" else False
    
    with open(os.path.join("cmu-wafer", datatype, filename)) as f:
        r = csv.reader(f, delimiter="\t", 
                       skipinitialspace=True, 
                       quoting=csv.QUOTE_NONE)
        # 데이터 컬럼은 리스트 1번째 아이템이므로(0번째는 index), dl 
        data = [row[1] for row in r]

    return (data, label)

In [59]:
# read_data("normal", "1551_05.11")

### How to read

* path : `$PATH/data/cmu-wafer/`
    * normal item path : `.../normal/(filename)`
    * abnormal item path : `.../abnormal/(filename)`
    * do not read files with an `.ann` extension

In [10]:
data = []
labels = []

for filename in os.listdir("cmu-wafer/normal"):
    if not ".ann" in filename:
        #print(filename)
        d, l = read_data("normal", filename)
        data.append(d)
        labels.append(l)

for filename in os.listdir("cmu-wafer/abnormal"):
    if not ".ann" in filename and not ".box" in filename:
        #print(filename)
        d, l = read_data("abnormal", filename)
        data.append(d)
        labels.append(l)


In [12]:
os.listdir("cmu-wafer/abnormal/")

['1549_01.11',
 '1549_01.12',
 '1549_01.15',
 '1549_01.6',
 '1549_01.7',
 '1549_01.8',
 '1549_01.ann',
 '1549_02.11',
 '1549_02.12',
 '1549_02.15',
 '1549_02.6',
 '1549_02.7',
 '1549_02.8',
 '1549_02.ann',
 '1549_04.11',
 '1549_04.12',
 '1549_04.15',
 '1549_04.6',
 '1549_04.7',
 '1549_04.8',
 '1549_04.ann',
 '1549_06.11',
 '1549_06.12',
 '1549_06.15',
 '1549_06.6',
 '1549_06.7',
 '1549_06.8',
 '1549_06.ann',
 '1549_07.11',
 '1549_07.12',
 '1549_07.15',
 '1549_07.6',
 '1549_07.7',
 '1549_07.8',
 '1549_07.ann',
 '1549_08.11',
 '1549_08.12',
 '1549_08.15',
 '1549_08.6',
 '1549_08.7',
 '1549_08.8',
 '1549_08.ann',
 '1559_12.11',
 '1559_12.12',
 '1559_12.15',
 '1559_12.6',
 '1559_12.7',
 '1559_12.8',
 '1559_12.ann',
 '1564_02.11',
 '1564_02.12',
 '1564_02.15',
 '1564_02.6',
 '1564_02.7',
 '1564_02.8',
 '1564_02.ann',
 '1564_02.box',
 '1565_21.11',
 '1565_21.12',
 '1565_21.15',
 '1565_21.6',
 '1565_21.7',
 '1565_21.8',
 '1565_21.ann',
 '1565_21.box',
 '1571_01.11',
 '1571_01.12',
 '1571_01.1

In [15]:
len(data) / 6

1194.0

In [49]:
def _read_cmu_wafer():
    def dtoi(i: int):
        # map desc to index
        # 6, 7, 8, 11, 12, 15를 리스트로
        return ['6', '7', '8', '11', '12', '15'].index(i)
    
    def parse_filename(filename: str):
        return tuple(re.split("\.", filename))

    def read_data(datatype: str, filename: str):
        assert datatype == "normal" or datatype == "abnormal"
        label = True if datatype == "normal" else False
        
        with open(os.path.join("cmu-wafer", datatype, filename)) as f:
            r = csv.reader(f, delimiter="\t", 
                           skipinitialspace=True, 
                           quoting=csv.QUOTE_NONE)
            data = [row[1] for row in r]
        return (data, label) 

    data = dict()
    labels = dict()

    for filename in os.listdir("cmu-wafer/normal"):
        if not ".ann" in filename:
            run_wafer, desc = parse_filename(filename)
            if not run_wafer in data.keys():
                data[run_wafer] = [None] * 6
                
            d, l = read_data("normal", filename)
            data[run_wafer][dtoi(desc)] = d
            labels[run_wafer] = l

    for filename in os.listdir("cmu-wafer/abnormal"):
        if not ".ann" in filename and not ".box" in filename:
            run_wafer, desc = parse_filename(filename)
            if not run_wafer in data.keys():
                data[run_wafer] = [None] * 6
            
            d, l = read_data("abnormal", filename)
            data[run_wafer][dtoi(desc)] = d
            labels[run_wafer] = l

    return data, labels

In [51]:
data, labels = _read_cmu_wafer()

In [57]:
import numpy as np
np.array(data.values())[0]

IndexError: too many indices for array